In [1]:
import json 


In [48]:


with open("/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/all_json/64814.json", "r") as file:
    cur_match = json.load(file)

In [49]:
# for ele in cur_match['innings'][0]['overs']:
cur_match

{'meta': {'data_version': '1.1.0', 'created': '2023-01-16', 'revision': 1},
 'info': {'balls_per_over': 6,
  'city': 'Napier',
  'dates': ['2002-12-29'],
  'event': {'name': 'India tour of New Zealand', 'match_number': 2},
  'gender': 'male',
  'match_type': 'ODI',
  'match_type_number': 1927,
  'officials': {'match_referees': ['RS Madugalle'],
   'tv_umpires': ['BF Bowden'],
   'umpires': ['EAR de Silva', 'DB Cowie']},
  'outcome': {'winner': 'New Zealand', 'by': {'runs': 35}},
  'overs': 50,
  'player_of_match': ['V Sehwag'],
  'players': {'New Zealand': ['SP Fleming',
    'NJ Astle',
    'MS Sinclair',
    'CD McMillan',
    'L Vincent',
    'JDP Oram',
    'BB McCullum',
    'KD Mills',
    'DL Vettori',
    'DR Tuffey',
    'PA Hitchcock'],
   'India': ['SC Ganguly',
    'V Sehwag',
    'VVS Laxman',
    'R Dravid',
    'Yuvraj Singh',
    'M Kaif',
    'SB Bangar',
    'Harbhajan Singh',
    'Z Khan',
    'J Srinath',
    'A Nehra']},
  'registry': {'people': {'A Nehra': '96fd40a

In [46]:
cur_match["info"]['gender']

{'balls_per_over': 6,
 'city': 'Napier',
 'dates': ['2002-12-29'],
 'event': {'name': 'India tour of New Zealand', 'match_number': 2},
 'gender': 'male',
 'match_type': 'ODI',
 'match_type_number': 1927,
 'officials': {'match_referees': ['RS Madugalle'],
  'tv_umpires': ['BF Bowden'],
  'umpires': ['EAR de Silva', 'DB Cowie']},
 'outcome': {'winner': 'New Zealand', 'by': {'runs': 35}},
 'overs': 50,
 'player_of_match': ['V Sehwag'],
 'players': {'New Zealand': ['SP Fleming',
   'NJ Astle',
   'MS Sinclair',
   'CD McMillan',
   'L Vincent',
   'JDP Oram',
   'BB McCullum',
   'KD Mills',
   'DL Vettori',
   'DR Tuffey',
   'PA Hitchcock'],
  'India': ['SC Ganguly',
   'V Sehwag',
   'VVS Laxman',
   'R Dravid',
   'Yuvraj Singh',
   'M Kaif',
   'SB Bangar',
   'Harbhajan Singh',
   'Z Khan',
   'J Srinath',
   'A Nehra']},
 'registry': {'people': {'A Nehra': '96fd40ae',
   'BB McCullum': 'b8a55852',
   'BF Bowden': 'efc0a862',
   'CD McMillan': 'f5390243',
   'DB Cowie': '36931357',
 

In [ ]:
import pandas as pd 

df = pd.read_csv("player_cur_match.csv")

In [35]:
df.head()

,identifier,full_name,batting_style,bowling_style,playing_role
0,bc005f0d,Aneka Akeilia Aresha White,Right hand Bat,Right arm Medium,NaN
1,5c67c7a6,Azam Ali Baig,NaN,NaN,NaN
2,6a58e069,Arun Bamal,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder
3,b4a23876,Alex Adrian Anthony Amsterdam,Left hand Bat,Right arm Offbreak,Top order Batter
4,482762af,Adewale A Adeoye,Right hand Bat,Right arm Medium,NaN


In [46]:
b_s = [str(style).split(',')[0] if isinstance(style, str) else 'Unknown' for style in df["bowling_style"].unique()]

In [52]:
# Categorize each style into filters
def classify_style(style):
    category = "swing/seam"

    if 'fast' in style.lower():
        category = "pace"
    elif any(spin_word in style.lower() for spin_word in ['spin', 'offbreak', 'legbreak', 'googly', 'orthodox', 'wrist']):
        category = "spin"
    elif style == "nan" :
        category = "nan"
    hand = "Right" if "Right" in style else "Left" if "Left" in style else "Unknown"
    return {"category": category, "dominant_hand": hand}

# Create dictionary mapping styles to categories
bowling_dict = {style: classify_style(style) for style in b_s}

# Print results
# print(bowling_dict)

In [49]:
bowling_dict

{'Right arm Medium': {'category': 'swing/seam', 'dominant_hand': 'Right'},
 'Unknown': {'category': 'swing/seam', 'dominant_hand': 'Unknown'},
 'Slow Left arm Orthodox': {'category': 'spin', 'dominant_hand': 'Left'},
 'Right arm Offbreak': {'category': 'spin', 'dominant_hand': 'Right'},
 'Right arm Medium fast': {'category': 'pace', 'dominant_hand': 'Right'},
 'Right arm Fast': {'category': 'pace', 'dominant_hand': 'Right'},
 'Left arm Medium fast': {'category': 'pace', 'dominant_hand': 'Left'},
 'Legbreak Googly': {'category': 'spin', 'dominant_hand': 'Unknown'},
 'Right arm Fast medium': {'category': 'pace', 'dominant_hand': 'Right'},
 'Left arm Medium': {'category': 'swing/seam', 'dominant_hand': 'Left'},
 'Legbreak': {'category': 'spin', 'dominant_hand': 'Unknown'},
 'Left arm Fast medium': {'category': 'pace', 'dominant_hand': 'Left'},
 'Left arm Wrist spin': {'category': 'spin', 'dominant_hand': 'Left'},
 'Right arm Bowler': {'category': 'swing/seam', 'dominant_hand': 'Right'},
 

In [ ]:
batter_dict = {}
prev_batter = -1
prev_category = -1
for over in cur_match['innings'][0]['overs']:
   for delivery in  over["deliveries"]:
      batter_name = delivery["batter"]
      batter_id = cur_match["info"]["registry"]["people"][batter_name]
      bowler_name = delivery["bowler"]
      bowler_id = cur_match["info"]["registry"]["people"][bowler_name]
      bw_style = df[df["identifier"]==bowler_id]["bowling_style"].values[0]
      b_s = str(bw_style).split(',')[0] if isinstance(bw_style, str) else "nan"
      # 
      classification = classify_style(b_s)
      category = classification["category"]
      runs = delivery.get("runs", {}).get("batter", 0)

      if batter_id not in batter_dict:
         batter_dict[batter_id] = {"batter_name": batter_name,"pace": 0, "pace_ball": 0, "swing/seam": 0, "swing/seam_ball": 0,"spin": 0, "spin_ball": 0,"nan" : 0, "nan_ball" : 0, "wicket" : "not_out"}
      batter_dict[batter_id][category] += runs
      batter_dict[batter_id][category+"_ball"] += 1
      if batter_id != prev_batter and prev_batter != -1:
          batter_dict[prev_batter]["wicket"] = prev_category
      prev_batter = batter_id
      prev_category = category

def safe_division(numerator, denominator, default=0):
    if denominator == 0:
        return default
    return round(numerator / denominator, 5)  # Round to 5 decimal places

final_dict = {}
for batter, stat in batter_dict.items():
    final_dict[batter] = {
        "pace": safe_division(stat["pace"], stat["pace_ball"]),
        "swing/seam": safe_division(stat['swing/seam'], stat['swing/seam_ball']),
        "spin": safe_division(stat['spin'], stat['spin_ball']),
        "nan": safe_division(stat['nan'], stat['nan_ball']),
        "wicket": stat['wicket']
    }

In [80]:
batter_dict

{'eea6b7f1': {'batter_name': 'SP Fleming',
  'pace': 1,
  'pace_ball': 6,
  'swing/seam': 0,
  'swing/seam_ball': 0,
  'spin': 0,
  'spin_ball': 0,
  'nan': 0,
  'nan_ball': 0},
 '99639abf': {'batter_name': 'NJ Astle',
  'pace': 57,
  'pace_ball': 71,
  'swing/seam': 4,
  'swing/seam_ball': 7,
  'spin': 15,
  'spin_ball': 19,
  'nan': 0,
  'nan_ball': 0},
 '1b668884': {'batter_name': 'MS Sinclair',
  'pace': 49,
  'pace_ball': 93,
  'swing/seam': 11,
  'swing/seam_ball': 13,
  'spin': 18,
  'spin_ball': 21,
  'nan': 0,
  'nan_ball': 0},
 'f5390243': {'batter_name': 'CD McMillan',
  'pace': 3,
  'pace_ball': 3,
  'swing/seam': 2,
  'swing/seam_ball': 4,
  'spin': 0,
  'spin_ball': 0,
  'nan': 0,
  'nan_ball': 0},
 '2764133a': {'batter_name': 'L Vincent',
  'pace': 19,
  'pace_ball': 18,
  'swing/seam': 6,
  'swing/seam_ball': 6,
  'spin': 9,
  'spin_ball': 8,
  'nan': 0,
  'nan_ball': 0},
 '3eac9d95': {'batter_name': 'JDP Oram',
  'pace': 1,
  'pace_ball': 4,
  'swing/seam': 0,
  'swing

In [81]:
final_dict

{'eea6b7f1': {'pace': 0.16667, 'swing/seam': 0, 'spin': 0, 'nan': 0},
 '99639abf': {'pace': 0.80282,
  'swing/seam': 0.57143,
  'spin': 0.78947,
  'nan': 0},
 '1b668884': {'pace': 0.52688,
  'swing/seam': 0.84615,
  'spin': 0.85714,
  'nan': 0},
 'f5390243': {'pace': 1.0, 'swing/seam': 0.5, 'spin': 0, 'nan': 0},
 '2764133a': {'pace': 1.05556, 'swing/seam': 1.0, 'spin': 1.125, 'nan': 0},
 '3eac9d95': {'pace': 0.25, 'swing/seam': 0, 'spin': 1.0, 'nan': 0},
 'b8a55852': {'pace': 0.25, 'swing/seam': 0, 'spin': 0.75, 'nan': 0},
 '7fb32e5b': {'pace': 0.5, 'swing/seam': 0, 'spin': 1.0, 'nan': 0},
 'd7c6af50': {'pace': 1.5, 'swing/seam': 0, 'spin': 0, 'nan': 0},
 '43936951': {'pace': 0.71429, 'swing/seam': 0, 'spin': 0, 'nan': 0},
 '27d1cc51': {'pace': 2.0, 'swing/seam': 0, 'spin': 0, 'nan': 0}}

In [ ]:
def get_batter_stat_per_match(json_path,player_data_path):

    with open(json_path, "r") as file:
        cur_match = json.load(file)
    df = pd.read_csv(player_data_path)
    final_dict = {}

    if cur_match['info']['gender'] != 'male' :
        return None 
    
    for innings in range(len(cur_match['innings'])):
        prev_batter = -1
        prev_category = -1
        for over in cur_match['innings'][innings]['overs']:
            for delivery in  over["deliveries"]:
                batter_name = delivery["batter"]
                batter_id = data["info"]["registry"]["people"][batter_name]
                bowler_name = delivery["bowler"]
                bowler_id = data["info"]["registry"]["people"][bowler_name]
                bw_style = df[df["identifier"]==bowler_id]["bowling_style"].values[0]
                b_s = str(bw_style).split(',')[0] if isinstance(bw_style, str) else "nan"
                # 
                classification = classify_style(b_s)
                category = classification["category"]
                runs = delivery.get("runs", {}).get("batter", 0)

                if batter_id not in batter_dict:
                    batter_dict[batter_id] = {"batter_name": batter_name,"pace": 0, "pace_ball": 0, "swing/seam": 0, "swing/seam_ball": 0,"spin": 0, "spin_ball": 0,"nan" : 0, "nan_ball" : 0, "wicket" : "not_out"}
                batter_dict[batter_id][category] += runs
                batter_dict[batter_id][category+"_ball"] += 1
                if batter_id != prev_batter and prev_batter != -1:
                    batter_dict[prev_batter]["wicket"] = prev_category
                prev_batter = batter_id
                prev_category = category

        def safe_division(numerator, denominator, default=0):
            if denominator == 0:
                return default
            return round(numerator / denominator, 5)  # Round to 5 decimal places

        for batter, stat in batter_dict.items():
            final_dict[batter] = {
                "pace": safe_division(stat["pace"], stat["pace_ball"]),
                "swing/seam": safe_division(stat['swing/seam'], stat['swing/seam_ball']),
                "spin": safe_division(stat['spin'], stat['spin_ball']),
                "nan": safe_division(stat['nan'], stat['nan_ball']),
                "wicket": stat['wicket']
            }
    return final_dict

## Bowler vs LHB & RHB 

In [2]:
import pandas as pd

import json

In [3]:
# Rough work 
csv_path = "/home/ai21btech11012/FIFS_dream11/src/data/processed/train/10_ODI.csv"
df = pd.read_csv(csv_path)

In [5]:
for match_id in df["match_id"].unique() :
    cur_match_df = df[df["match_id"]==match_id]

    # Create batsmen vs opponent bowler data
    for 


    # Create bowler vs opponent batsmen data

51388

In [3]:
def classify_bat_style(bat_style):
    category = 'nan'
    if 'left' in bat_style.lower():
        category = 'LHB'
    elif 'right' in bat_style.lower():
        category = 'RHB'
    return category

def get_bowler_stat_per_match(json_path,player_data_path) :

    with open(json_path, "r") as file:
        cur_match = json.load(file)
    df = pd.read_csv(player_data_path)

    if cur_match['info']['gender'] != 'male' :
        return None 
    bowler_dict = {}
    for innings in range(len(cur_match['innings'])):
        LHB_set = set()
        RHB_set = set()
        for over in cur_match['innings'][innings]['overs']:
            for delivery in  over["deliveries"]:
                bowler_name = delivery["bowler"]
                bowler_id = cur_match["info"]["registry"]["people"][bowler_name]
                batter_name = delivery["batter"]
                batter_id = cur_match["info"]["registry"]["people"][batter_name]
                try : 
                 bt_style = df[df["identifier"]==batter_id]["batting_style"].values[0]
                except :
                    print(json_path)
                    print("Batter ID is not available in player_data csv ",batter_id,batter_name)
                    
                final_bt_style = classify_bat_style(bt_style)
                if final_bt_style == 'LHB':
                    LHB_set.add(batter_id)
                elif final_bt_style == 'RHB':
                    RHB_set.add(batter_id)
                runs = delivery.get("runs", {}).get("total", 0)
                is_wicket = "wickets" in delivery

                if bowler_id not in bowler_dict:
                    bowler_dict[bowler_id] = {"LHB": 0, "RHB" : 0 ,"LHB_count" : 0,"RHB_count" : 0, "LHB_balls": 0, "RHB_balls": 0, "LHB_Wickets": 0, "RHB_Wickets": 0,"LHB_dot": 0, "RHB_dot": 0}

                bowler_dict[bowler_id][final_bt_style] += runs
                bowler_dict[bowler_id][final_bt_style+"_balls"] += 1
                bowler_dict[bowler_id][final_bt_style+"_Wickets"] += is_wicket
                bowler_dict[bowler_id][final_bt_style+"_dot"] += 1 if runs == 0 else 0
        # print(LHB_set)
        # print(RHB_set)
        for _ , val in bowler_dict.items():
            val["LHB_count"] = len(LHB_set)
            val["RHB_count"] = len(RHB_set)
            
    return bowler_dict


In [15]:
match_path = "/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/all_json/64814.json"
player_data_path = "/home/ai21btech11012/FIFS_dream11/src/data/raw/additional_data/player_data.csv"

bw_dict = get_bowler_stat_per_match(match_path,player_data_path)

{'eea6b7f1', 'd7c6af50', '3eac9d95'}
{'1b668884', 'b8a55852', 'f5390243', '7fb32e5b', '99639abf', '43936951', '2764133a', '27d1cc51'}
{'725529bc', '1c914163'}
{'8ba8195d', 'bad31fac', 'd84378a4', '8b5b6769', '91a4a398', '0604ef16', '0184dc35', 'de8cce37'}


In [16]:
# bw_dict 

In [4]:
import os
from joblib import Parallel, delayed
from collections import defaultdict

player_data_path = "/home/ai21btech11012/FIFS_dream11/src/data/raw/additional_data/player_data.csv"
json_folder = "/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/all_json"

all_files = os.listdir(json_folder)
all_paths = [os.path.join(json_folder, file) for file in all_files]

# Function to process a single file
def process_match(match_path):
    bw_dict = get_bowler_stat_per_match(match_path, player_data_path)
    # bt_dict = get_batter_stat_per_match(match_path, player_data_path)
    if bw_dict is None:
        return None
    match_id = os.path.basename(match_path).split(".")[0]
    return match_id, bw_dict

# Run parallel processing
results = Parallel(n_jobs=-1)(delayed(process_match)(path) for path in all_paths)

# Aggregate the results
overall_bowler_dict = defaultdict(dict)
for result in results:
    if result is None:
        continue
    match_id, bw_dict = result
    for bowler_id, stats in bw_dict.items():
        overall_bowler_dict[bowler_id][match_id] = stats

In [5]:
# overall_bowler_dict
# create a csv from the overall_bowler_dict


# Convert overall_bowler_dict to a flat list of dictionaries
data = []
for bowler_id, matches in overall_bowler_dict.items():
    for match_id, stats in matches.items():
        row = {"bowler_id": bowler_id, "match_id": match_id, **stats}
        data.append(row)

# Convert list to DataFrame
df = pd.DataFrame(data)

# Save as CSV
output_path = "/home/ai21btech11012/FIFS_dream11/src/data/processed/bowler_vs_LHB_RHB.csv"
df.to_csv(output_path, index=False)

print(f"CSV file saved at {output_path}")

CSV file saved at /home/ai21btech11012/FIFS_dream11/src/data/processed/bowler_vs_LHB_RHB.csv
